# Introducción para la optimización en la toma de decisiones
- Alumna: Rosa Huamaní Pucho - 20200422

## Ejercicio 1: el problema de la dieta
---


In [1]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSq9X74urGAB_5n_MIJ9ZGIboKSvBdokVTBXVLh_qqZnmLRTJioOF431Rzys3Qi9UaFwWXjeq6Wmd5/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

### Paso 1: inciciamos el modelo

In [3]:
import pulp as pp

In [4]:
model = pp.LpProblem(name='dieta-problem', # just the name
                     sense=pp.LpMinimize) # type of problem

### Paso 2: declarar las VARIABLES

In [6]:
# cuanto de VegaVita?
VegaVita = pp.LpVariable(name="VegaVita",  
                    lowBound=0,  
                    cat='Continuous')

# cuanto de HappyHealth?
HappyHealth = pp.LpVariable(name="HappyHealth",
                 lowBound=0,
                 cat='Continuous')

### Paso 3: creamos la función para optimizar

In [7]:
VegaVitaCoeff=0.2
HappyHealthCoeff=0.3
obj_func = VegaVitaCoeff*VegaVita + HappyHealthCoeff*HappyHealth

### Paso 4: representamos las restricciones

In [ ]:
C1= pp.LpConstraint(name='vitaminac Constraint',   # just the name
                    e= 20*VegaVita + 30*HappyHealth, rhs=60, # linear combination of constraint and rhs
                    sense=pp.LpConstraintGE) 
C2= pp.LpConstraint(name='calcio Constraint',   
                    e= 500*VegaVita + 250*HappyHealth, rhs=1000, 
                    sense=pp.LpConstraintGE) 
C3= pp.LpConstraint(name='hierro Constraint',   
                    e= 9*VegaVita + 2*HappyHealth, rhs=18, 
                    sense=pp.LpConstraintGE) 
C4= pp.LpConstraint(name='niacina Constraint',   
                    e= 2*VegaVita + 10*HappyHealth, rhs=20, 
                    sense=pp.LpConstraintGE)
C5= pp.LpConstraint(name='magnesio Constraint',   
                    e= 60*VegaVita + 90*HappyHealth, rhs=360, 
                    sense=pp.LpConstraintGE)  

### Paso 5: Construyendo el modelo

In [9]:
model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

### Paso 6: resolviendo el modelo

In [10]:
solver_list = pp.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


In [11]:
#solverToUse = pp.COIN_CMD(msg=False)
model.solve(0);

In [12]:
import pandas as pd

Results={"Model Status":pp.LpStatus[model.status]}
Results.update({"Optimal Solution":pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
Results

{'Model Status': 'Optimal',
 'Optimal Solution': 1.2,
 'HappyHealth': 3.1304348,
 'VegaVita': 1.3043478}

In [13]:
#or
pd.DataFrame.from_dict(Results,orient='index').T.set_index('Model Status').style.format('{:,}')

,Optimal Solution,HappyHealth,VegaVita
Model Status,,,
Optimal,1.2,3.1304348,1.3043478


**Respuesta** : Para satisfacer los requerimientos nutricionales mínimos y gastar lo menos posible, el modelo sugiere adquirir aproximadamente 3.13 tabletas de HappyHealth y 1.30 tabletas de VegaVita, lo cual costaría $1.2 en total.

## Ejercicio 2: el problema de la dieta
---